# MLI Bank Fraud Prediction Neural Network

In [8]:
import pandas as pd

In [9]:
mydf=pd.read_csv("/machine_learning_india_ai_challenge-dataset/CLEANED Training data - Python.csv")

In [11]:
x=mydf.drop(columns=['Class'])#'Index','Class'])
y=mydf['Class']

In [12]:
from imblearn.combine import SMOTETomek
smk=SMOTETomek()
x_res,y_res=smk.fit_sample(x,y)

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x_res,y_res,test_size=0.2)

# Neural Network Time!

In [14]:
import tensorflow as tf

In [86]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(xtrain.shape[1], )), 
  tf.keras.layers.Dense(512, activation='softmax'),
  tf.keras.layers.Dropout(0.2), #1/Epochs, prevent overfitting 
  tf.keras.layers.Dense(58, activation='softmax'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='sigmoid')
])

In [87]:
model.compile(optimizer='adam', #Training procedure
              loss='sparse_categorical_crossentropy', #What to minmize
              metrics=['accuracy'])

In [88]:
model.fit(xtrain, ytrain, epochs=5) #Epoch= no.times to test each image

Epoch 1/5
12324/12324 [==============================] - 29s 2ms/step - loss: 0.1309 - accuracy: 0.9661
Epoch 2/5
12324/12324 [==============================] - 29s 2ms/step - loss: 0.0477 - accuracy: 0.9838
Epoch 3/5
12324/12324 [==============================] - 28s 2ms/step - loss: 0.0394 - accuracy: 0.9867
Epoch 4/5
12324/12324 [==============================] - 29s 2ms/step - loss: 0.0324 - accuracy: 0.9890
Epoch 5/5
12324/12324 [==============================] - 29s 2ms/step - loss: 0.0300 - accuracy: 0.9901


In [89]:
model.evaluate(xtest, ytest)

preds=model.predict(xtest)

3081/3081 [==============================] - 4s 1ms/step - loss: 0.0140 - accuracy: 0.9938


In [90]:
realpreds=[]
for row in preds:
    if row[0]>row[1]:
        realpreds.append(0)
    else:
        realpreds.append(1)
print(len(realpreds))

98586


In [91]:
print(len(ytest))

98586


In [92]:
confmat=tf.math.confusion_matrix(ytest,realpreds)
print("Confusion matrix:\n",confmat)

aucroc=tf.keras.metrics.AUC(num_thresholds=200, curve='ROC',summation_method='interpolation')
aucroc.update_state(ytest,realpreds)
import numpy
print("AUROC:")
print(aucroc.result().numpy())


Confusion matrix:
 tf.Tensor(
[[48875   411]
 [  199 49101]], shape=(2, 2), dtype=int32)
AUROC:
0.9938122


# Exam time!

In [93]:
exam=pd.read_csv("/machine_learning_india_ai_challenge-dataset/TEST.csv")

In [94]:
examx=exam.drop(columns=['Index'])

In [95]:
answers=model.predict(examx)

In [96]:
realanswers=[]
for row in answers:
    if row[0]>row[1]:
        realanswers.append(0)
    else:
        realanswers.append(1)
print(len(realanswers))

ones=0
for i in realanswers:
    if i ==1:
        ones+=1
print(ones)


352
53


In [97]:
myfile=open("/machine_learning_india_ai_challenge-dataset/Submission10-Cleaned+NN2.csv","w")
myfile.write("Index,Class\n")
for i in range(len(realanswers)):
    myfile.write(str(i)+","+str(realanswers[i])+"\n")
myfile.close()